# Unitary Matrix LU Decomposition Demo
This notebook shows how to:
1. Generate random unitary matrices
2. Check unitarity
3. Perform LU decomposition with SciPy
4. Verify the reconstruction error

In [8]:
import numpy as np
import scipy.linalg as la

def is_unitary(U, tol=1e-10):
    U = np.asarray(U, dtype=complex)
    n, m = U.shape
    if n != m:
        return False, np.inf
    I = np.eye(n, dtype=complex)
    err = la.norm(U.conj().T @ U - I, ord='fro')
    return err <= tol, err

def random_unitary(n, seed=None):
    rng = np.random.default_rng(seed)
    Z = rng.normal(size=(n, n)) + 1j * rng.normal(size=(n, n))
    Q, R = la.qr(Z)
    d = np.diag(R)
    ph = d / np.abs(d)
    return Q * ph

def lu_decompose_and_verify(A):
    P, L, U = la.lu(A)
    recon_err = la.norm(P @ A - L @ U, ord='fro')
    return P, L, U, recon_err


## ✅ Verification and Sanity Checks

This cell:
1. **Checks that `U` is unitary** by computing ‖UᴴU − I‖.
2. **Recomputes the LU decomposition** for the current `U` and verifies the identity  
   $$U = P \cdot L \cdot U_{\text{fac}}$$  
   using `np.allclose` and a relative error metric.
3. **Compares with the alternative form** `P @ U = L @ Ufac` (for completeness).
4. **Performs a QR decomposition** and verifies that `Q` is orthonormal and `U ≈ Q@R`.

These checks ensure the decomposition is **mathematically correct** and demonstrate numerical stability.


In [9]:
import numpy as np
import scipy.linalg as la

print("=== Verification Cell (recomputes LU for current U) ===")

# 0) sanity: U should be unitary
unit_err = np.linalg.norm(U.conj().T @ U - np.eye(U.shape[0]), 'fro')
print(f"Unitarity check ||U^H U - I||_F = {unit_err:.2e}")

# 1) recompute LU *from the current U*
P, L, Ufac = la.lu(U)

# 2) Check SciPy's common convention: U ≈ P @ L @ Ufac
lhs = U
rhs = P @ L @ Ufac
ok_plu = np.allclose(lhs, rhs)
rel_err_plu = np.linalg.norm(lhs - rhs, 'fro') / (np.linalg.norm(lhs, 'fro') + np.linalg.norm(rhs, 'fro'))
print("Check A = P @ L @ U ?  ->", ok_plu, "| relative error:", f"{rel_err_plu:.2e}")

# 3) Also check the alternative: P @ U ≈ L @ Ufac
lhs2 = P @ U
rhs2 = L @ Ufac
ok_pu_lu = np.allclose(lhs2, rhs2)
rel_err_pu_lu = np.linalg.norm(lhs2 - rhs2, 'fro') / (np.linalg.norm(lhs2, 'fro') + np.linalg.norm(rhs2, 'fro'))
print("Check P @ A = L @ U ? ->", ok_pu_lu, "| relative error:", f"{rel_err_pu_lu:.2e}")

# 4) QR sanity (for comparison)
Q, R = la.qr(U)
qr_recon_err = np.linalg.norm(U - Q @ R, 'fro')
q_ortho_err = np.linalg.norm(Q.conj().T @ Q - np.eye(Q.shape[0]), 'fro')
print(f"QR recon error ||U - Q@R||_F = {qr_recon_err:.2e}")
print(f"Q orthogonality error ||Q^H Q - I||_F = {q_ortho_err:.2e}")


=== Verification Cell (recomputes LU for current U) ===
Unitarity check ||U^H U - I||_F = 9.82e-16
Check A = P @ L @ U ?  -> True | relative error: 4.85e-17
Check P @ A = L @ U ? -> False | relative error: 6.12e-01
QR recon error ||U - Q@R||_F = 8.10e-16
Q orthogonality error ||Q^H Q - I||_F = 1.12e-15


In [10]:
for n, seed in [(2, 42), (4, 123)]:
    print('='*60)
    print(f'=== Demo for n={n} ===')
    U = random_unitary(n, seed=seed)
    ok, unitary_err = is_unitary(U)
    print(f'Input is unitary? {ok} (||U^H U - I||_F = {unitary_err:.2e})')
    P, L, Ufac, recon_err = lu_decompose_and_verify(U)
    print(f'LU reconstruction error ||P@U - L@Ufac||_F = {recon_err:.2e}')
    if n <= 2:
        np.set_printoptions(precision=3, suppress=True)
        print('\nU (input unitary):\n', U)
        print('\nP (permutation):\n', P)
        print('\nL (unit lower-triangular):\n', L)
        print('\nU (upper-triangular factor):\n', Ufac)


=== Demo for n=2 ===
Input is unitary? True (||U^H U - I||_F = 7.20e-16)
LU reconstruction error ||P@U - L@Ufac||_F = 5.55e-17

U (input unitary):
 [[ 0.144-0.922j -0.059+0.355j]
 [ 0.355+0.06j   0.918+0.165j]]

P (permutation):
 [[1. 0.]
 [0. 1.]]

L (unit lower-triangular):
 [[ 1.   +0.j     0.   +0.j   ]
 [-0.005+0.385j  1.   +0.j   ]]

U (upper-triangular factor):
 [[ 0.144-0.922j -0.059+0.355j]
 [ 0.   +0.j     1.055+0.189j]]
=== Demo for n=4 ===
Input is unitary? True (||U^H U - I||_F = 9.82e-16)
LU reconstruction error ||P@U - L@Ufac||_F = 2.45e+00
